#Banco de Dados

In [4]:
import pandas as pd
import numpy as np

colunas = open('colunas_vinho.txt', 'r')
nome_colunas = [coluna[:-1] for coluna in colunas]
colunas.close()
bd_vinho = pd.read_csv('banco_vinho.csv',names=nome_colunas, header=None)
bd_vinho.head()

,Vinho,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,Diluted wines,Prolin
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


#Executar Rede Neural

In [ ]:
from arquiteturas_rede_neural.arquiteturas import *
from bancos_de_dados.bd_tratado import *
import time
import numpy as np

arquitetura = criar_arquitetura_vinho()
num_epocas = 500
learning_rate = 1
num_execucoes = 10

rede = arquitetura
rede.learning_rate = learning_rate
rede.insere_sinapses_e_bias()
coluna_alvo = rede.banco.columns[rede.atributos_de_saida[0]]  # continente, Vinho, vidro, iris
banco = rede.banco
tempos = np.array([])

#Cria os vetores que armazenarão as acuracias referentes a cada sub-banco
acuracia_treino = np.zeros(num_execucoes)
acuracia_teste = np.zeros(num_execucoes)
acuracia_total = np.zeros(num_execucoes)
for execucao in range(num_execucoes):
    print('\nexecução : ', execucao)
    tempo_inicial = time.perf_counter()
    #cria uma nova rede
    rede = arquitetura
    rede.learning_rate = learning_rate
    rede.insere_sinapses_e_bias()

    #define as bases de treino e teste
    base_treino, base_teste, tipos_saidas = tratar_bd(banco, coluna_alvo)
    base_treino = base_treino.sample(frac=1).reset_index(drop=True)

    #Realiza o backpropagation na função aprender
    rede.aprender(num_epocas, base_treino)

    #Realiza o teste de cada base e retorna as matrizes de confusão
    matriz_treino = rede.testar(tipos_saidas, base_treino)
    matriz_teste = rede.testar(tipos_saidas, base_teste)
    matriz_total = rede.testar(tipos_saidas, banco)

    tempos = np.append(tempos, time.perf_counter() - tempo_inicial)

    #Utiliza a função calcula resultados para usar as matrizes de confusão e 
    #coletar os dados referentes a sensibilidade, confiabilidades etc...
    print('base de treino: ')
    acuracia_treino[execucao] = calcula_resultados(matriz_treino, True)
    print('base de teste: ')
    acuracia_teste[execucao] = calcula_resultados(matriz_teste, True)
    print('base inteira: ')
    acuracia_total[execucao] = calcula_resultados(matriz_total, True)
print('')
print('media de treino: ', np.mean(acuracia_treino))
print('media de teste: ', np.mean(acuracia_teste))
print('media de total: ', np.mean(acuracia_total))
print('')
print('desvio padrão de treino: ', np.std(acuracia_treino))
print('desvio padrão de teste: ', np.std(acuracia_teste))
print('desvio padrão de total: ', np.std(acuracia_total))
print()
print('Tempo médio de execução: ', np.mean(tempos))



# Objeto Camada

In [5]:
import numpy as np
#Objeto camada, representando todas as possíveis camadas da arquitetura,
#seja ela de entrada, oculta ou final. Possui atributos  para armazenar
#valores importantes como número de neurônios, os valores presentes nos
#neurônios, as sinapses que a conectam com a próxima camada, o bias e o
#erro que são utilizados no Feedfoward e no Backpropagation.
class Camada:
    def __init__(self, numero_neuronios, final=False):
        self.numero_neuronios = numero_neuronios
        self.neuronios = np.zeros((numero_neuronios, 1))
        self.sinapses = None
        self.final = final
        self.bias = np.zeros((numero_neuronios, 1))
        self.erro = None

#Este método serve para que possamos modificar o número de neurônios da
#camada, como alguns atributos dependem dessa variável, criamos este
#método para atualizar tanto o número de neurônios, quanto outros atributos
#que dependem dele.
    def atualiza_numero_de_neuronios(self, numero_neuronios):
        self.numero_neuronios = numero_neuronios
        self.neuronios = np.zeros((numero_neuronios, 1))
        self.bias = np.zeros((numero_neuronios, 1))

#Objeto Rede Neural

In [7]:
import numpy as np
import pandas as pd
from camada import Camada
from funcoes_uteis import sigmoide
from funcoes_uteis import multiplicar_matrizes
from funcoes_uteis import derivar_sigmoide
from funcoes_uteis import tahn
from ferramentas import *

#O Objeto Rede Neural, onde ficarão armazenadas as camadas, especificações do
#banco de dados como número de atributos de entrada e saída, o próprio banco
#para utilização pode parte dela, e outros parâmetros de execução e informações
#importantes como a linha atual que a rede está lendo do banco, o valor esperado
#para a classe dessa linha entre outros.
class Rede_Neural:
    def __init__(self, atributos_entradas, atributos_saidas,  num_camadas, neuronios_por_camada_oculta, banco):
        self.atributos_de_entrada = atributos_entradas
        self.atributos_de_saida = atributos_saidas
        self.banco = banco
        self.num_entradas = self.atributos_de_entrada.size
        self.num_saidas = self.atributos_de_saida.size
        self.numero_camadas = num_camadas
        self.neuronios_por_camada_oculta = neuronios_por_camada_oculta
        self.camadas = self.cria_camadas()
        self.valor_esperado = None
        self.learning_rate = 1
        self.linha_atual = None

#Este método em essência serve para verificar se as camadas foram criadas de
#forma correta, ele mostra na tela várias informações referentes a todas as
#camadas da rede neural.
    def mostra_informacoes_das_camadas(self):
        print('')
        for camada in range(self.numero_camadas):
            print("Informações da camada %d: " % camada)
            print("Numero de neuronios: ", self.camadas[camada].numero_neuronios)
            print("Camada final?: ", self.camadas[camada].final)
            print("Sinapses da camada: ")
            print(self.camadas[camada].sinapses, "\n")

#Este método é chamado no momento de criação da rede neural, ao mesmo tempo
#ele irá criar as camadas conforme a arquitetura definida no arquivo de
#execução do algoritmo.
    def cria_camadas(self):
        camadas = []
        for camada in range(self.numero_camadas):
            if camada == 0:
                camadas.append(Camada(self.num_entradas))
            elif camada == self.numero_camadas-1:
                camadas.append(Camada(self.num_saidas))
            else:
                camadas.append(Camada(self.neuronios_por_camada_oculta))
        camadas[-1].final = True
        return camadas

#Este método deve ser chamado imediatamente após a criação da rede, ele irá
#inserir todas as sinapses entre todos os neurônios de todas as camadas, além
#disso, ele irá inserir o vetor de bias que será utilizado no Feedfoward e
#Backpropagation, as sinapses (pesos) são criados de forma aleatória, com 
#valores entre 0 e 1 para cada peso.
    def insere_sinapses_e_bias(self):
        for camada in range(self.numero_camadas):
            if not self.camadas[camada].final:
                camada_1 = self.camadas[camada]
                camada_2 = self.camadas[camada+1]
                linhas = camada_2.numero_neuronios
                colunas = camada_1.numero_neuronios
                sinapses = np.random.rand(linhas, colunas)
                self.camadas[camada].sinapses = np.copy(sinapses)
                self.camadas[camada].bias = np.zeros((self.camadas[camada + 1].numero_neuronios, 1))

#Este método é responsável por inserir os atributos da linha que será lida na
#primeira camada, por isso seu único parâmetro é a linha do banco que será lida.
    def inserir_entradas(self, linha):
        banco = self.banco.values
        for entrada in range(self.num_entradas):
            atributo = self.atributos_de_entrada[entrada]
            self.camadas[0].neuronios[entrada] = sigmoide(banco[linha][atributo])

#Este método definirá qual será o valor esperado, ou seja, o valor correto da
#classe que possui os atributos na camada de entrada, note que este valor
#já foi convertido para seu equivalente na função Sigmoide.
    def inserir_saidas(self, linha):
        banco = self.banco.values
        for saida in range(self.num_saidas):
            atributo = self.atributos_de_saida[saida]
            valor = banco[linha][atributo]
            #print(valor)
            if valor == 'Iris-setosa':
                valor = 1
            elif valor == 'Iris-versicolor':
                valor = 2
            elif valor == 'Iris-virginica':
                valor = 3
            else:
                valor = valor
            self.valor_esperado = sigmoide(valor)

#O método feedfoward fará os calculos necessários utilizando os valores na
#camada de entrada para que se chegue ao valor de saída da rede, armazenado
#na última camada.
    def feedfoward(self):
        #print("lendo linha: ", self.linha_atual)
        for camada_atual in range(self.numero_camadas-1):
            camada = self.camadas[camada_atual]
            if not camada.final:
                multiplicacao_matricial = multiplicar_matrizes(camada.sinapses, camada.neuronios)
                multiplicacao_matricial = multiplicacao_matricial + camada.bias
                self.camadas[camada_atual + 1].neuronios = sigmoide(multiplicacao_matricial)
                '''print('sinapses ligadas a camada %d: ' % camada_atual)
                print(camada.sinapses)
                print('neuronios da camada atual: ')
                print(camada.neuronios)
                print('multiplicação das sinapses pelos neuronios: ')
                print(self.camadas[camada_atual+1].neuronios)'''


#O método a seguir serve para testar o feedfoward uma única ves em uma linha
#do bando de dados, isso é importante para que possamos fazer verificações
#únicas no bando depois que a rede já aprendeu, ou caso queiramos executar
#o feedfoward várias vezes já inserindo as entradas e saídas podemos chamar
#essa função em loop.
    def testar_feed_foward(self, linha, banco):
        self.banco = banco
        self.inserir_entradas(linha)
        self.inserir_saidas(linha)
        self.linha_atual = linha
        self.feedfoward()

#Função para propagar o erro obtido na saída da rede neural, modificando os 
#pesos das sinapses entre as camadas, além de modificar o Bias
    def backpropagation(self):
        erro_saida = self.valor_esperado - self.camadas[-1].neuronios
        derivada_saida = derivar_sigmoide(self.camadas[-1].neuronios)
        transposta_oculto = np.transpose(self.camadas[-2].neuronios)

        gradiente = np.multiply(derivada_saida, erro_saida)
        gradiente = gradiente * self.learning_rate

        self.camadas[-2].bias = self.camadas[-2].bias + gradiente

        delta_pesos_oculto_saida = np.matmul(gradiente, transposta_oculto)
        self.camadas[-2].sinapses = self.camadas[-2].sinapses + delta_pesos_oculto_saida
        self.camadas[-1].erro = erro_saida

        for i in range(self.numero_camadas - 2, 0, -1):
            transposta_pesos = np.transpose(self.camadas[i].sinapses)
            erro = np.matmul(transposta_pesos, self.camadas[i+1].erro)
            derivada = derivar_sigmoide(self.camadas[i].neuronios)
            transposta = np.transpose(self.camadas[i-1].neuronios)
    
            gradiente_O = np.multiply(erro, derivada)
            gradiente_O = gradiente_O * self.learning_rate
    
            self.camadas[i-1].bias = self.camadas[i-1].bias + gradiente_O
    
            delta_pesos = np.matmul(gradiente_O, transposta)
            self.camadas[i-1].sinapses = self.camadas[i-1].sinapses + delta_pesos
            self.camadas[i].erro = erro

#Função para testar o algoritmo após o aprendizado, apenas classificando as 
#amostras sem modificar os seus parâmetros
    def aprender(self, num_epocas, base_treino):
        self.banco = base_treino
        num_linhas = len(base_treino)
        for epocas in range(num_epocas):
            for linha in range(num_linhas):
                self.linha_atual = linha
                self.inserir_entradas(linha)
                self.inserir_saidas(linha)
                self.feedfoward()
                self.backpropagation()


#Função que executa o aprendizado do algoritmo utilizando a base de treino. 
#A mesma utiliza outras funções, como o backpropagation() e o feedfoward()
    def testar(self, tipos_saidas, base_teste):
        matriz_confusao = np.zeros((len(tipos_saidas), len(tipos_saidas)))
        num_linhas = len(base_teste)
        resultado = None
        base_numpy = base_teste.values

        for linha in range(num_linhas):
            self.testar_feed_foward(linha, base_teste)

            valor_1 = sigmoide(tipos_saidas[0])
            valor_2 = sigmoide(tipos_saidas[1])
            valor_ultimo = sigmoide(tipos_saidas[-1])
            valor_penultimo = sigmoide(tipos_saidas[-2])

            if (len(tipos_saidas) == 2):
                if self.camadas[-1].neuronios[0][0] < ((valor_1 + valor_2) / 2):
                    resultado = np.where(tipos_saidas == tipos_saidas[0])
                elif self.camadas[-1].neuronios[0][0] >= ((valor_1 + valor_2) / 2):
                    resultado = np.where(tipos_saidas == tipos_saidas[-1])
            else:
                if self.camadas[-1].neuronios[0][0] < ((valor_1 + valor_2) / 2):
                    resultado = np.where(tipos_saidas == tipos_saidas[0])

                elif self.camadas[-1].neuronios[0][0] >= ((valor_1 + valor_2) / 2) and self.camadas[-1].neuronios[0][0] < ((valor_ultimo + valor_penultimo) / 2):
                    for j in range(1, len(tipos_saidas) - 1):
                        valor_1 = sigmoide(tipos_saidas[j])
                        valor_2 = sigmoide(tipos_saidas[j - 1])
                        valor_3 = sigmoide(tipos_saidas[j + 1])
                        if (self.camadas[-1].neuronios[0][0] >= (valor_1 + valor_2)/2) and (self.camadas[-1].neuronios[0][0] < (valor_1 + valor_3))/2:
                            resultado = np.where(tipos_saidas == tipos_saidas[j])

                elif self.camadas[-1].neuronios[0][0] >= ((valor_ultimo + valor_penultimo) / 2):
                    resultado = np.where(tipos_saidas == tipos_saidas[-1])

            valor_teste = np.where(tipos_saidas == base_numpy[linha][self.atributos_de_saida[0]])
            matriz_confusao[int(valor_teste[0])][int(resultado[0])] += 1
        print(matriz_confusao)
        return matriz_confusao

# Funções do arquivo Funções Úteis

In [8]:
#A função sigmoide é a principal função de ativação do algoritmo de rede neural,
#ela será utilizada tanto na etapa de aprendizado quanto nas etapas de teste.
def sigmoide(x):
    return 1/(1+np.exp(-x))

#A derivada da função de ativação é necessária na etapa de propagação do erro,
#ou seja, na função Backpropagation.
def derivar_sigmoide(x):
    return x * (1 - x)

#A função tahn é outra função de ativação presente na literatura, porém, não 
#utilizada neste trabalho efetivamente, apenas foi colocada aqui como exemplo.
def tahn(x):
    return 2 / (1 + np.exp(-2*x)) - 1

#A função multiplica matrizes simplesmente recebe duas matrizes e as multiplica
#utilizando um método da biblioteca Numpy.
def multiplicar_matrizes(a, b):
    return np.matmul(a, b)

#Ferramentas

In [ ]:
import numpy as np

#calcula e retorna uma base de treino única
def retorna_treino(base):
    treino = base.sample(frac=0.7)
    return treino

#calcula e retorna uma base de teste única
def retorna_teste(base, base_treino):
    teste = base.drop(base_treino.index)
    return teste

#calcula sensibilidade, confiabilidades entre outros a partir da matriz de
#confusão
def calcula_resultados(matriz, verbose=False):
    
    num_classes = len(matriz)

    sensibilidade = np.zeros([num_classes])
    especificidade = np.zeros([num_classes])
    confiabilidade_positiva = np.zeros([num_classes])
    confiabilidade_negativa = np.zeros([num_classes])

    tp = np.zeros([num_classes])
    tn = np.zeros([num_classes])
    fn = np.zeros([num_classes])
    fp = np.zeros([num_classes])

    acertos = 0
    acuracia = 0
    total = 0

    for classe in range(num_classes):
        for linha in range(num_classes):
            for coluna in range(num_classes):
                if classe == linha == coluna:
                    tp[classe] += matriz[linha][coluna]
                elif classe != linha == coluna:
                    tn[classe] += matriz[linha][coluna]
                elif classe == linha != coluna:
                    fn[classe] += matriz[linha][coluna]
                elif classe == coluna != linha:
                    fp[classe] += matriz[linha][coluna]

    if verbose:
        print('TP =', tp)
        print('TN =', tn)
        print('FN =', fn)
        print('FP =', fp)

    for linha in range(num_classes):
        for coluna in range(num_classes):
            if linha == coluna:
                acertos += matriz[linha][coluna]
            total += matriz[linha][coluna]

    acuracia = (acertos*100)/total

    for classe in range(num_classes):
        sensibilidade[classe] = tp[classe]/(tp[classe] + fn[classe])
        especificidade[classe] = tn[classe]/(tn[classe] + fp[classe])
        confiabilidade_positiva[classe] = tp[classe]/(tp[classe] + fp[classe])
        confiabilidade_negativa[classe] = tn[classe]/(tn[classe] + fn[classe])
        
        if tp[classe] + fn[classe] == 0:
            sensibilidade[classe] = 0
        if tn[classe] + fp[classe] == 0:
            especificidade[classe] = 0
        if tp[classe] + fp[classe] == 0:
            confiabilidade_positiva[classe] = 0
        if tn[classe] + fn[classe] == 0:
            confiabilidade_negativa[classe] = 0

        if verbose:
            print('----------- Classe %d -----------' %(classe+1))
            print('Sensibilidade: ', sensibilidade[classe])
            print('Especificidade: ', especificidade[classe])
            print('Confiabilidade Positiva: ', confiabilidade_positiva[classe])
            print('Confiabilidade Negativa: ', confiabilidade_negativa[classe])
    if verbose:
        print('Media da Sensibilidade: ', np.mean(sensibilidade))
        print('Media da Especificidade: ', np.mean(especificidade))
        print('Media da Confiabilidade Positiva: ', np.mean(confiabilidade_positiva))
        print('Media da Confiabilidade Negativa: ', np.mean(confiabilidade_negativa))

    if verbose:
        print('Acurácia:', acuracia, end='\n')

    return acuracia